## 데이터 전처리

아래의 데이터를 이용해 
- LPOINT_BIG_COMP_01_DEMO.csv
- LPOINT_BIG_COMP_02_PDDE.csv
- LPOINT_BIG_COMP_03_COP_U.csv
- LPOINT_BIG_COMP_04_PD_CLAC.csv
- LPOINT_BIG_COMP_05_BR.csv
- LPOINT_BIG_COMP_06_LPAY.csv

다음 데이터를 만들었다.
- data1.csv
- data2.csv
- data3.csv
- data4.csv
- data5.csv
- data6.csv

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [7]:
import os
sorted(os.listdir("../3. LOTTO_데이터 및 모델 세이브 파일/data"))

['LPOINT_BIG_COMP_01_DEMO.csv',
 'LPOINT_BIG_COMP_02_PDDE.csv',
 'LPOINT_BIG_COMP_03_COP_U.csv',
 'LPOINT_BIG_COMP_04_PD_CLAC.csv',
 'LPOINT_BIG_COMP_05_BR.csv',
 'LPOINT_BIG_COMP_06_LPAY.csv']

In [8]:
for i, file in enumerate(sorted(os.listdir("../3. LOTTO_데이터 및 모델 세이브 파일/data"))):
    globals()[f'data{i+1}'] = pd.read_csv("../3. LOTTO_데이터 및 모델 세이브 파일/data/"+file)
datas = [data1, data2, data3, data4, data5, data6]

### 이상치 처리

In [9]:
cust_outlier = ['M057015266','M919374790','M223630342','M785169881']

for cust in cust_outlier:
    data1 = data1.drop(data1[data1['cust']==cust].index)
    data2 = data2.drop(data2[data2['cust']==cust].index)
    data3 = data3.drop(data3[data3['cust']==cust].index)
    data6 = data6.drop(data6[data6['cust']==cust].index)

### 결측치 처리

In [10]:
# 결측치

# data2
null_idx = [(i, data2[(data2['br_c'].isnull())&(data2['cop_c']==i)].index)
            for i in data2[data2['br_c'].isnull()]['cop_c'].value_counts().index]

for _ in null_idx:
    data2.loc[_[1],'br_c'] = _[0]+'00000'

    
# data3
data3['br_c'] = data3['br_c'].fillna('E0100000')


# data5
add = pd.DataFrame({
    'br_c':['A0200000','A0300000','A0600000','A0100000','A0500000','E0100000'],
    'cop_c':['A02','A03','A06','A01','A05','E01'],
    'zon_hlv':['Z00','Z00','Z00','Z00','Z00','Z00'],
    'zon_mcls':['Z00000','Z00000','Z00000','Z00000','Z00000','Z00000']
})

data5 = pd.concat([data5,add])

### 중복 제거

In [11]:
drop_dup = data2.groupby(['cust','rct_no','cop_c','br_c','pd_c','de_dt','de_hr'])['buy_am','buy_ct'].sum()
drop_dup

buy_am  buy_ct
cust       rct_no       cop_c br_c     pd_c   de_dt    de_hr                  
M000034966 A01016688590 A01   A010045  PD0777 20210116 18      78000.0       2
                                       PD0796 20210116 18        100.0       1
           A01016718167 A01   A010045  PD0816 20210116 18      62100.0       1
           A02357398521 A02   A020116  PD0616 20210325 13      17900.0       2
                                       PD0630 20210325 13      59700.0       3
...                                                                ...     ...
M999962961 E02024505549 A02   A0200000 PD1441 20210515 10       1180.0       1
           E06002810292 A06   A0600000 PD0959 20210113 18     129000.0       1
           E06004609359 A06   A0600000 PD0954 20210122 14      34300.0       1
           E06025413888 A06   A0600000 PD0967 20210406 17      89000.0       1
           E06035713248 A06   A0600000 PD1889 20210503 22      50000.0       1

[3710275 rows x 2 columns]

In [12]:
drop_dup.reset_index(inplace=True)

In [13]:
drop_dup

,cust,rct_no,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct
0,M000034966,A01016688590,A01,A010045,PD0777,20210116,18,78000.0,2
1,M000034966,A01016688590,A01,A010045,PD0796,20210116,18,100.0,1
2,M000034966,A01016718167,A01,A010045,PD0816,20210116,18,62100.0,1
3,M000034966,A02357398521,A02,A020116,PD0616,20210325,13,17900.0,2
4,M000034966,A02357398521,A02,A020116,PD0630,20210325,13,59700.0,3
...,...,...,...,...,...,...,...,...,...
3710270,M999962961,E02024505549,A02,A0200000,PD1441,20210515,10,1180.0,1
3710271,M999962961,E06002810292,A06,A0600000,PD0959,20210113,18,129000.0,1
3710272,M999962961,E06004609359,A06,A0600000,PD0954,20210122,14,34300.0,1
3710273,M999962961,E06025413888,A06,A0600000,PD0967,20210406,17,89000.0,1


## Only Data(Lpay 사용여부)

아래의 데이터를 이용해 
- data2.csv
- data3.csv
- data6.csv


다음 데이터를 만들었다.
- only_data2.csv
- only_data3.csv
- only_data6.csv

**data2와 data3 중 Lpay로 결제한 경우는 data6에 포함되어 있으므로 Lpay사용여부를 구분하고자 한다.**
1. 엘페이를 이용하지 않고 유통사를 이용한 고객의 거래정보(only_data2)
2. 엘페이를 이용하지 않고 제휴사를 이용한 고객의 거래정보(only_data3)
3. 엘페이를 이용한 고객의 거래정보(only_data6)<br>
를 구하고자 한다.<br><br>
- 이를 확인하기 위해 중복처리를 하지 않은 데이터를 사용해야 한다.
- 현재 data2는 중복처리를 한 데이터이므로 중복처리하지 않은 원본 데이터를 불러와야 한다.

In [8]:
import os
os.listdir("../3. LOTTO_데이터 및 모델 세이브 파일/data_preprocessing")

['data1.csv', 'data2.csv', 'data3.csv', 'data4.csv', 'data5.csv', 'data6.csv']

In [15]:
for i, file in enumerate(os.listdir("../3. LOTTO_데이터 및 모델 세이브 파일/data_preprocessing")):
    globals()[f'data{i+1}'] = pd.read_csv("../3. LOTTO_데이터 및 모델 세이브 파일/data_preprocessing/"+file,index_col=0)

In [16]:
real_data2=pd.read_csv('../3. LOTTO_데이터 및 모델 세이브 파일/data/LPOINT_BIG_COMP_02_PDDE.csv')

In [17]:
#이상치 제거
cust_outlier = ['M057015266','M919374790','M223630342','M785169881']

for cust in cust_outlier:
    real_data2 = real_data2.drop(real_data2[real_data2['cust']==cust].index)

In [18]:
# 결측값 채우기
null_idx = [(i, real_data2[(real_data2['br_c'].isnull())&(real_data2['cop_c']==i)].index)
            for i in real_data2[real_data2['br_c'].isnull()]['cop_c'].value_counts().index]

for _ in null_idx:
    real_data2.loc[_[1],'br_c'] = _[0]+'00000'

In [19]:
real_data2['kind']='two'
data3['kind']='three'
data6['kind']='six'

### Only_data2
**엘페이를 이용하지 않고 유통사를 이용한 고객의 거래정보**

In [20]:
int_df26 = pd.merge(real_data2, data6[['cust', 'cop_c', 'chnl_dv', 'de_dt', 'de_hr', 'buy_am','kind']], on=['cust', 'de_dt', 'de_hr', 'cop_c', 'chnl_dv','buy_am'],how='outer',indicator=True)
int_df26.head()

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,kind_x,kind_y,_merge
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1.0,two,NaN,left_only
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1.0,two,NaN,left_only
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,two,NaN,left_only
3,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,two,NaN,left_only
4,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1.0,two,NaN,left_only


In [21]:
Only_data2=int_df26.loc[int_df26._merge=='left_only']
Only_data2.drop(['kind_x','kind_y','_merge'],axis=1,inplace=True)
Only_data2.head()

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1.0
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1.0
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1.0
3,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1.0
4,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1.0


### Only_data3
**엘페이를 이용하지 않고 제휴사를 이용한 고객의 거래정보**

In [22]:
int_df36 = pd.merge(data3, data6[['cust', 'cop_c', 'chnl_dv', 'de_dt', 'de_hr', 'buy_am','kind']], on=['cust', 'de_dt', 'de_hr', 'cop_c', 'chnl_dv','buy_am'],how='outer',indicator=True)
int_df36.head()

,cust,rct_no,cop_c,br_c,chnl_dv,de_dt,vst_dt,de_hr,buy_am,kind_x,kind_y,_merge
0,M839993508,21102612B015763935,B01,B010012,1,20211026,20211026.0,12,60000,three,NaN,left_only
1,M839993508,21110610B014219744,B01,B010012,1,20211106,20211106.0,10,17100,three,NaN,left_only
2,M839993508,21021112B013419710,B01,B010012,1,20210211,20210211.0,12,136500,three,NaN,left_only
3,M839993508,21092010B012637545,B01,B010012,1,20210920,20210920.0,10,34200,three,NaN,left_only
4,M839993508,21101009D015920171,D01,D010614,1,20211010,20211010.0,9,2500,three,NaN,left_only


In [23]:
Only_data3=int_df36.loc[int_df36._merge=='left_only']
Only_data3.drop(['kind_x','kind_y','_merge'],axis=1,inplace=True)
Only_data3.head()

,cust,rct_no,cop_c,br_c,chnl_dv,de_dt,vst_dt,de_hr,buy_am
0,M839993508,21102612B015763935,B01,B010012,1,20211026,20211026.0,12,60000
1,M839993508,21110610B014219744,B01,B010012,1,20211106,20211106.0,10,17100
2,M839993508,21021112B013419710,B01,B010012,1,20210211,20210211.0,12,136500
3,M839993508,21092010B012637545,B01,B010012,1,20210920,20210920.0,10,34200
4,M839993508,21101009D015920171,D01,D010614,1,20211010,20211010.0,9,2500


### 중복확인

In [24]:
dup = Only_data2.duplicated()
print('중복된 데이터의 개수',len(dup[dup==True]))
print('중복되지 않은 데이터의 개수:',len(dup[dup==False]))

중복된 데이터의 개수 236127
중복되지 않은 ㅁ데이터의 개수: 4086286


In [25]:
dup = Only_data3.duplicated()
print('중복된 데이터의 개수',len(dup[dup==True]))
print('중복되지 않은 데이터의 개수:',len(dup[dup==False]))

중복된 데이터의 개수 0
중복되지 않은 데이터의 개수: 226919


In [26]:
dup = data6.duplicated()
print('중복된 데이터의 개수',len(dup[dup==True]))
print('중복되지 않은 데이터의 개수:',len(dup[dup==False]))

중복된 데이터의 개수 0
중복되지 않은 데이터의 개수: 350662


- Only_data2에서만 중복된 데이터가 존재한다.
- 구매금액, 구매수량을 제외한 모든 항목이 중복된 데이터를 **"제거"가 아닌 "병합"을 통해** 중복처리를 진행

In [27]:
Only_data2_중복처리 = Only_data2.groupby(list(Only_data2.columns[:-2]))['buy_am','buy_ct'].sum()
Only_data2_중복처리.reset_index(inplace=True)
Only_data2_중복처리

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct
0,M000034966,A01016688590,1,A01,A010045,PD0777,20210116,18,78000.0,2.0
1,M000034966,A01016688590,1,A01,A010045,PD0796,20210116,18,100.0,1.0
2,M000034966,A01016718167,1,A01,A010045,PD0816,20210116,18,62100.0,1.0
3,M000034966,A02357398521,1,A02,A020116,PD0616,20210325,13,17900.0,2.0
4,M000034966,A02357398521,1,A02,A020116,PD0630,20210325,13,59700.0,3.0
...,...,...,...,...,...,...,...,...,...,...
3673769,M999962961,E02024505549,2,A02,A0200000,PD1441,20210515,10,1180.0,1.0
3673770,M999962961,E06002810292,2,A06,A0600000,PD0959,20210113,18,129000.0,1.0
3673771,M999962961,E06004609359,2,A06,A0600000,PD0954,20210122,14,34300.0,1.0
3673772,M999962961,E06025413888,2,A06,A0600000,PD0967,20210406,17,89000.0,1.0


In [28]:
dup = Only_data2_중복처리.duplicated()
print('중복된 데이터의 개수',len(dup[dup==True]))
print('중복되지 않은 데이터의 개수:',len(dup[dup==False]))

중복된 데이터의 개수 0
중복되지 않은 데이터의 개수: 3673774
